---
# Praxisteil 4 - Metaheuristiken
---

---
## 1. Iterated Greedy
---

---
### 1.1 Wiederholung allgemeiner Ablauf
---

Ablauf Iterated Greedy

![AblaufIG](../DataFiles/IteratedGreedyAblauf.png)

----
### 1.2 Startlösung
----
NEH-Heuristik

In [2]:
! pip install deap

  Using cached deap-1.3.1-cp39-cp39-win_amd64.whl (108 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] 拒绝访问。: 'E:\\anaconda\\Lib\\site-packages\\deap'
Consider using the `--user` option or check the permissions.



In [3]:
import deap

ModuleNotFoundError: No module named 'deap'

In [1]:
from Solver import *

data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json")

solver = Solver(data, 1060) #1014, 754 |  1060, 736

startSolution = solver.ConstructionPhase("NEH")

currentSolution = deepcopy(startSolution)


ModuleNotFoundError: No module named 'deap'

----
### 1.3 Destruktion
----
Entferne zufällig _numberJobsToRemove_ Aufträge aus aktueller Lösung. Dabei ist _numberJobsToRemove_ ein Parameter des Iterated Greedy.
Als Ergebnis ensteht die Menge entfernter Aufträge _removedJobs_ und eine unvollständige Lösung _partialPermutation_.

In [ ]:
numberJobsToRemove = 2 # 1st parameter



----
### 1.4 Konstruktion
----
Füge die _removedJobs_ unter Beachtung der Reihenfolge an der jeweils besten Position (NEH) wieder zu _partialPermutation_ hinzu und gib die neue vollständige Lösung zurück. 

----
### 1.5 Akzeptanz und Gedächtnis
---- 
Akzeptiere neue Lösung als aktuelle Lösung, wenn
* neue Lösung besser als aktuelle Lösung (immer)
* Akzeptanzkriterium erfüllt (zufällig)

Akzeptanzkriterium bei Ruiz/ Stützle:  $\pi''$ - neue Lösung (_newSolution_), $\pi$ - aktuelle Lösung (_currentSolution_)
$$
    \begin{equation}
    e^{- \frac{C_{\max}\left( \pi'' \right) - C_{\max}\left( \pi \right)}{Temperature}} \text{ mit } Temperature = T \cdot \frac{\sum Bearbeitungszeiten}{n \cdot m \cdot 10}
    \end{equation}
$$  
$T$ ist der 2. Parameter des Iterated Greedy. Je höher $T$ ist, desto größer ist die Wahrscheinlichkeit, schlechtere Lösungen zu akzeptieren.

Neue beste Lösung wird im _SolutionPool_ gespeichert.

In [ ]:
print(f'Solution before checking for acceptance: \n{currentSolution}')

baseTemperature = 1 # 2nd parameter

        
print(f'Solution after checking for acceptance: \n{currentSolution}')

Akzeptanzkriterium in Funktion für bessere Übersichtlichkeit

In [ ]:
def AcceptWorseSolution(currentObjectiveValue, newObjectiveValue):  
    temperature = baseTemperature * solver.InputData.TotalProcessingTime / (solver.InputData.n * solver.InputData.m * 10)
    probability = math.exp(-(newObjectiveValue - currentObjectiveValue) / temperature)

    return solver.RNG.random() <= probability

----
### 1.6 Lokale Suche
----
Verfahren der lokalen Suche können optional verwendet werden, um die aktuelle Lösung zu verbessern. Häufig wird __Iterative Improvement__ mit der Insertion-Nachbarschaft verwendet.

In [ ]:
print(f'Solution before Local Search:\n{currentSolution}')



print(f'Solution after nach Local Search\n{currentSolution}')

----
### 1.7 Iterated Greedy als Steuerungsmechanismus der Komponenten
----
Iterated Greedy ist iterative Abfolge von Destruktion und Konstruktion $\Rightarrow$ wird in Schleife immer wieder durchgeführt bis Stoppkriterium erreicht ist. 

Hier im Beispiel ist Stoppkriterium die Anzahl an Iterationen _maxIterations_, aber bspw. auch Zeitlimit oder Anzahl Iterationen ohne Verbesserungen möglich.

In [ ]:
maxIterations = 10 # stop criterion number of iterations


Iterated Greedy ausführen

In [ ]:
solver = Solver(data, 1060)

startSolution = solver.ConstructionPhase("NEH")

bestSolution = Run(startSolution)

print(f'Best solution found.\n{bestSolution}')

---
## 2. Klasse für Iterated Greedy
---

Analog wie für den Algorithmus _IterativeImprovement_ soll jetzt auch für _IteratedGreedy_ eine eigene Klasse angelegt werden, damit eine Instanz der Klasse an den Solver übergeben werden kann.

_Iterated Greedy_ soll wie _IterativeImprovement_ von _ImprovementAlgorithm_ erben.

Notwendige __Parameter__ sind Attribute:
* NumberJobsToRemove
* BaseTemperature
* MaxIterations
* Instanz des Local Search Algorithmus

EvaluationLogic, SolutionPool und random number generator werden von Solver an Algorithmus übergeben.

__Programmieraufgabe:__ Ergänzen sie die Klasse _IteratedGreedy_, sodass sie an den Solver als Algorithmus übergeben werden kann.

__Funktionen:__ Konstruktor, Initialize, Destruction, Construction, AcceptWorseSolution, Run

In [ ]:
class IteratedGreedy(ImprovementAlgorithm):
    def __init__(self, inputData, numberJobsToRemove, baseTemperature, maxIterations, localSearchAlgorithm=None):
        super().__init__(inputData)

  

Der Algorithmus kann jetzt in _RunLocalSearch()_ an den Solver übergeben und ausgeführt werden.

In [ ]:
    data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json") #"../TestInstancesJson/Small/VFR40_10_3_SIST.json"

    insertion = IterativeImprovement(data, 'BestImprovement', ['Insertion'])
    taillardInsertion = IterativeImprovement(data, 'BestImprovement', ['TaillardInsertion'])
    iteratedGreedy = IteratedGreedy(
        data, 
        numberJobsToRemove=2, 
        baseTemperature=1, 
        maxIterations=100)

    solver = Solver(data, 1010)

    solver.RunLocalSearch(
        constructiveSolutionMethod='NEH',
        algorithm=iteratedGreedy)

----
## 3. Exkurs: DEAP und Genetic Algorithms
----

Distributed Evolutionary Algorithms in Python ([DEAP](https://github.com/DEAP/deap)) ist ein Framework für Evolutionary Computation, mit u.a. den populationsbasierten Metaheuristiken Genetic Algorithms und Particle Swarm Optimization.

![Metaheuristiken](../DataFiles/MH.png)

### Wichtige Begriffe für Genetic Algorithms:
* Individuum: Einzelne Lösung
* Population: Sammlung von Individuen
* Generation: Population in einer Iteration
* Reproduktion: Erzeugung neuer Individuen aus bekannten Individuen
* Selektion: Auswahl von Individuen für Reproduktion und neue Generation
* Fitness: Bewertung eines Individuums (Zielfunktionswert)
* Mutation: Zufällige Veränderung eines Individuums

![GeneticAlgorithm](../DataFiles/GA.png)

----
### Beispiel DEAP
----

__Creator__

Meta-Factory um Klassen zu erzeugen

Parameters:
* name – The name of the class to create.
* base – A base class from which to inherit.
* attribute – One or more attributes to add on instantiation of this class, optional.


In [ ]:
from Solver import *

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

data = InputData("../TestInstancesJson/Small/VFR10_5_5_SIST.json")

solver = Solver(data, 1011)

creator.create("FitnessMin", base.Fitness, weights=[-1.0]) # minimization problem, weights as list
creator.create("Individual", list, fitness=creator.FitnessMin) # single individual stored in python list

__Toolbox__

Erzeugt Konfiguration des Algorithmus

In [ ]:
toolbox = base.Toolbox() 

# Individual is a permutation of integer indices
toolbox.register("indices", solver.RNG.choice, range(solver.InputData.n), solver.InputData.n, replace=False)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

# Population is a collection of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
pop = toolbox.population(100) # 100 individuals in population

# Operators
toolbox.register("mate", tools.cxPartialyMatched) # set crossover 
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # set mutation
toolbox.register("select", tools.selTournament, tournsize=3) # set selection mechanism

__Fitness-Funktion__

Bewertung eines Indivduums

In [ ]:
def EvalPFSP(solver, individual):
    solution = Solution(solver.InputData.InputJobs, individual)
    solver.EvaluationLogic.DefineStartEnd(solution)
    return [solution.Makespan]

toolbox.register("evaluate", EvalPFSP, solver) # fitness function

__Statistiken__

Sammlung von Werten während Laufzeit des Algorithmus'

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

__Hall of fame__

Gedächtnis zum Speichern der besten Lösungen; hier nur beste Lösung

In [ ]:
hof = tools.HallOfFame(1)

__Ausführen__

In [ ]:
random.seed(solver.Seed)
algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=0.8, mutpb=0.2, ngen=100, stats=stats, halloffame=hof)

bestPermutation = hof[0]
bestSolution = Solution(solver.InputData.InputJobs, bestPermutation)
solver.EvaluationLogic.DefineStartEnd(bestSolution)
print(f'Best found Solution.\n {bestSolution}')

----
#### Funktion für Solver
----

Parameter als Argumente
* Populationsgröße _populationSize_
* Anzahl Generationen _generations_
* Rekombinationswahrscheinlichkeit _matingProb_
* Mutationswahrscheinlichkeit _mutationProb_

In [ ]:
def EvalPFSP(self, individual):
    solution = Solution(self.InputData.InputJobs, individual)
    self.EvaluationLogic.DefineStartEnd(solution)
    return [solution.Makespan]

def RunGeneticAlgorithm(self, populationSize, generations, matingProb, mutationProb):
    # Creator - meta-factory to create new classes
    creator.create("FitnessMin", base.Fitness, weights=[-1.0])
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox() 

    # Individual is a permutation of integer indices
    toolbox.register("indices", solver.RNG.choice, range(self.InputData.n), self.InputData.n, replace=False)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

    # Population is a collection of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    pop = toolbox.population(populationSize) # number of individuals in population

    # Operators
    toolbox.register("mate", tools.cxPartialyMatched) # set crossover 
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05) # set mutation
    toolbox.register("select", tools.selTournament, tournsize=3) # set selection mechanism
    
    # Fitness function
    toolbox.register("evaluate", self.EvalPFSP) 
    
    # Statistics during run time
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
            
    # Hall of fame --> Best individual
    hof = tools.HallOfFame(1)

    random.seed(self.Seed)
    algorithms.eaSimple(population=pop, toolbox=toolbox, cxpb=matingProb, mutpb=mutationProb, ngen=generations, stats=stats, halloffame=hof)

    bestPermutation = hof[0]
    bestSolution = Solution(self.InputData.InputJobs, bestPermutation)
    self.EvaluationLogic.DefineStartEnd(bestSolution)
    print(f'Best found Solution.\n {bestSolution}')

Solver.EvalPFSP = EvalPFSP
Solver.RunGeneticAlgorithm = RunGeneticAlgorithm

Vergleich Iterated Greedy und Genetic Algorithm

In [ ]:
data = InputData("../TestInstancesJson/Large/VFR100_20_1_SIST.json") # TestInstances/Small/VFR40_10_3_SIST.txt 

taillardInsertion = IterativeImprovement(data, 'FirstImprovement', neighborhoodTypes=['TaillardInsertion'])
iteratedGreedy = IteratedGreedy(
    data, 
    numberJobsToRemove=2, 
    baseTemperature=1, 
    maxIterations=50, 
    localSearchAlgorithm=taillardInsertion)

solver = Solver(data, 1010)

solver.RunLocalSearch(
    constructiveSolutionMethod='NEH',
    algorithm=iteratedGreedy)
    
solver.RunGeneticAlgorithm(
    populationSize=100, 
    generations=200, 
    matingProb=0.8, 
    mutationProb=0.2)
